In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install pyyaml==5.1

import torch
TORCH_VERSION = ".".join(torch.__version__.split(".")[:2])
CUDA_VERSION = torch.__version__.split("+")[-1]
print("torch: ", TORCH_VERSION, "; cuda: ", CUDA_VERSION)

# Install detectron2 that matches the above pytorch version
# See https://detectron2.readthedocs.io/tutorials/install.html for instructions
!pip install detectron2 -f https://dl.fbaipublicfiles.com/detectron2/wheels/$CUDA_VERSION/torch$TORCH_VERSION/index.html

torch:  1.10 ; cuda:  cu111
Looking in links: https://dl.fbaipublicfiles.com/detectron2/wheels/cu111/torch1.10/index.html


In [ ]:
import detectron2
from detectron2.utils.logger import setup_logger

# Setup detectron2 logger
setup_logger()

# import some common libraries
import numpy as np
import os, json, cv2, random
from google.colab.patches import cv2_imshow

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog

In [ ]:
!curl -L "https://app.roboflow.com/ds/UFzKFrkvDS?key=wFV1LglJaf" > roboflow.zip; unzip roboflow.zip; rm roboflow.zip

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   887  100   887    0     0   1825      0 --:--:-- --:--:-- --:--:--  1821
100 45.6M  100 45.6M    0     0  51.5M      0 --:--:-- --:--:-- --:--:-- 51.5M
Archive:  roboflow.zip
 extracting: README.dataset.txt      
 extracting: README.roboflow.txt     
   creating: test/
 extracting: test/A00_00_bmp_jpg.rf.1ea980a7e2c95288e0fb2943975ad14e.jpg  
 extracting: test/A00_00_bmp_jpg.rf.3f0408d1e7637b1db780e7388effc433.jpg  
 extracting: test/A00_00_bmp_jpg.rf.5b6e263c867e9695556755c7fee765f8.jpg  
 extracting: test/A00_00_bmp_jpg.rf.8e6d38741a15c3a9b335dc4799a15221.jpg  
 extracting: test/A00_00_bmp_jpg.rf.c251c869ab047f762f1f01468a370cee.jpg  
 extracting: test/A00_08_bmp_jpg.rf.48e718e06371696548670065f72111c4.jpg  
 extracting: test/A00_08_bmp_jpg.rf.e3e4ae3adcc0095cea53cc8b60f4a812.jpg  
 extracting: test/A01_04_bmp_jpg.rf.02ab0

In [ ]:
from detectron2.data.datasets import register_coco_instances

register_coco_instances("dataset_train", {}, "/content/train/_annotations.coco.json", "/content/train")
register_coco_instances("dataset_test", {}, "/content/test/_annotations.coco.json", "/content/test")

In [ ]:
from detectron2.engine import DefaultTrainer
from detectron2.evaluation import COCOEvaluator

class CocoTrainer(DefaultTrainer):

  @classmethod
  def build_evaluator(cls, cfg, dataset_name, output_folder=None):

    if output_folder is None:
        os.makedirs("coco_eval", exist_ok=True)
        output_folder = "coco_eval"

    return COCOEvaluator(dataset_name, cfg, False, output_folder)

In [ ]:
import os
from detectron2.config import get_cfg

cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_X_101_32x8d_FPN_3x.yaml"))
cfg.DATASETS.TRAIN = ("dataset_train",)
cfg.DATASETS.TEST  = ("dataset_train",)
#cfg.DATASETS.TEST = ("my_dataset_val",)

cfg.DATALOADER.NUM_WORKERS = 4
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-Detection/faster_rcnn_X_101_32x8d_FPN_3x.yaml")  # Let training initialize from model zoo
#cfg.MODEL.WEIGHTS = "/content/drive/MyDrive/Projects/Final Year Project/Saved_Model [512 x 512 Epochs-500]/model_final.pth"
cfg.SOLVER.IMS_PER_BATCH = 4
cfg.SOLVER.BASE_LR = 0.001

cfg.SOLVER.WARMUP_ITERS = 120
cfg.SOLVER.MAX_ITER = 1200 #adjust up if val mAP is still rising, adjust down if overfit
# cfg.SOLVER.STEPS = []
cfg.SOLVER.GAMMA = 0.05

cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 5
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 2 #your number of classes + 1

os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
trainer = CocoTrainer(cfg)
trainer.resume_or_load(resume=False)
trainer.train()

[03/10 20:32:43 d2.engine.defaults]: Model:
GeneralizedRCNN(
  (backbone): FPN(
    (fpn_lateral2): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral3): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral4): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral5): Conv2d(2048, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output5): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (top_block): LastLevelMaxPool()
    (bottom_up): ResNet(
      (stem): BasicStem(
        (conv1): Conv2d(
          3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False
          (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
        )
      )
      (res

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
model_final_68b088.pkl: 421MB [00:07, 58.0MB/s]                           
Skip loading parameter 'roi_heads.box_predictor.cls_score.weight' to the model due to incompatible shapes: (81, 1024) in the checkpoint but (3, 1024) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.cls_score.bias' to the model due to incompatible shapes: (81,) in the checkpoint but (3,) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.b

[03/10 20:32:57 d2.engine.train_loop]: Starting training from iteration 0


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
/usr/local/lib/python3.7/dist-packages/detectron2/structures/image_list.py:88: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  max_size = (max_size + (stride - 1)) // stride * stride
/usr/local/lib/python3.7/dist-p

[03/10 20:35:12 d2.utils.events]:  eta: 2:15:50  iter: 19  total_loss: 1.399  loss_cls: 0.8084  loss_box_reg: 0.3516  loss_rpn_cls: 0.2228  loss_rpn_loc: 0.01102  time: 6.7200  data_time: 0.0352  lr: 0.00015917  max_mem: 8668M
[03/10 20:37:26 d2.utils.events]:  eta: 2:07:35  iter: 39  total_loss: 1.226  loss_cls: 0.4574  loss_box_reg: 0.6904  loss_rpn_cls: 0.04591  loss_rpn_loc: 0.00893  time: 6.6951  data_time: 0.0168  lr: 0.00032567  max_mem: 8668M
[03/10 20:39:43 d2.utils.events]:  eta: 2:17:07  iter: 59  total_loss: 1.063  loss_cls: 0.3441  loss_box_reg: 0.719  loss_rpn_cls: 0.01355  loss_rpn_loc: 0.009893  time: 6.7568  data_time: 0.0168  lr: 0.00049217  max_mem: 8668M
[03/10 20:41:58 d2.utils.events]:  eta: 2:14:51  iter: 79  total_loss: 0.9519  loss_cls: 0.2737  loss_box_reg: 0.6453  loss_rpn_cls: 0.02094  loss_rpn_loc: 0.007836  time: 6.7487  data_time: 0.0157  lr: 0.00065867  max_mem: 8668M
[03/10 20:44:12 d2.utils.events]:  eta: 2:12:27  iter: 99  total_loss: 0.9172  loss_cls

In [ ]:
#test evaluation
from detectron2.data import DatasetCatalog, MetadataCatalog, build_detection_test_loader
from detectron2.evaluation import COCOEvaluator, inference_on_dataset

cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.85
predictor = DefaultPredictor(cfg)
evaluator = COCOEvaluator("dataset_test", cfg, False, output_dir="./output/")
val_loader = build_detection_test_loader(cfg, "dataset_test")
inference_on_dataset(trainer.model, val_loader, evaluator)

WARNING [03/10 23:00:40 d2.evaluation.coco_evaluation]: COCO Evaluator instantiated using config, this is deprecated behavior. Please pass in explicit arguments instead.
WARNING [03/10 23:00:40 d2.data.datasets.coco]: 
Category ids in annotations are not in [1, #categories]! We'll apply a mapping for you.

[03/10 23:00:40 d2.data.datasets.coco]: Loaded 153 images in COCO format from /content/test/_annotations.coco.json
[03/10 23:00:40 d2.data.build]: Distribution of instances among all 2 categories:
|   category    | #instances   |  category  | #instances   |
|:-------------:|:-------------|:----------:|:-------------|
| Mitotic-cells | 0            |  mitotic   | 216          |
|               |              |            |              |
|     total     | 216          |            |              |
[03/10 23:00:40 d2.data.dataset_mapper]: [DatasetMapper] Augmentations used in inference: [ResizeShortestEdge(short_edge_length=(800, 800), max_size=1333, sample_style='choice')]
[03/10 23:0

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
/usr/local/lib/python3.7/dist-packages/detectron2/structures/image_list.py:88: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  max_size = (max_size + (stride - 1)) // stride * stride


[03/10 23:00:52 d2.evaluation.evaluator]: Inference done 11/153. Dataloading: 0.0017 s/iter. Inference: 1.0884 s/iter. Eval: 0.0003 s/iter. Total: 1.0904 s/iter. ETA=0:02:34
[03/10 23:00:58 d2.evaluation.evaluator]: Inference done 16/153. Dataloading: 0.0020 s/iter. Inference: 1.0895 s/iter. Eval: 0.0003 s/iter. Total: 1.0920 s/iter. ETA=0:02:29
[03/10 23:01:03 d2.evaluation.evaluator]: Inference done 21/153. Dataloading: 0.0023 s/iter. Inference: 1.0910 s/iter. Eval: 0.0003 s/iter. Total: 1.0939 s/iter. ETA=0:02:24
[03/10 23:01:08 d2.evaluation.evaluator]: Inference done 26/153. Dataloading: 0.0025 s/iter. Inference: 1.0895 s/iter. Eval: 0.0003 s/iter. Total: 1.0926 s/iter. ETA=0:02:18
[03/10 23:01:14 d2.evaluation.evaluator]: Inference done 31/153. Dataloading: 0.0027 s/iter. Inference: 1.0895 s/iter. Eval: 0.0003 s/iter. Total: 1.0929 s/iter. ETA=0:02:13
[03/10 23:01:19 d2.evaluation.evaluator]: Inference done 36/153. Dataloading: 0.0029 s/iter. Inference: 1.0895 s/iter. Eval: 0.000

OrderedDict([('bbox',
              {'AP': 34.86920482326941,
               'AP-Mitotic-cells': nan,
               'AP-mitotic': 34.86920482326941,
               'AP50': 74.49233671631,
               'AP75': 22.019871723020728,
               'APl': nan,
               'APm': 37.51756386648014,
               'APs': 8.284307470781913})])